In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#cp /kaggle/input/model-trainingdata/df_final-2023_01_01_10_42_1672569751 .

In [ ]:
#ls -l

In [ ]:

#dbfile = open('df_final-2023_01_01_10_42_1672569751', 'rb') 
#dataset = pickle.load(dbfile)

In [ ]:
#dbfile.close()

In [ ]:
#from sklearn.linear_model import Perceptron

In [ ]:
# extracting signal/noise label feature
#y=dataset['y']

In [ ]:
#temp_dataset=dataset[:]

In [ ]:
# removing y from dataset
#x=temp_dataset.drop(['y'], axis=1)

In [ ]:
# perceptron model
#ppn = Perceptron(eta0=0.1, random_state=1)

In [ ]:
# x = temp dataset 
# y = label 
#ppn.fit(x,y)

In [ ]:
#!touch perceptron_ppn-`date +"%Y_%m_%d_%I_%M_%s"`

In [ ]:
# code for saving the perceptron model ppn
#model_file = open('perceptron_ppn-2023_01_01_12_30_1672576249','ab')
#pickle.dump(ppn,model_file)     

In [ ]:
#model_file.close()

In [ ]:
#ls -l

# Test Data Generation

In [ ]:
!pip install git+https://github.com/PyFstat/PyFstat@python37

In [ ]:
import numpy as np

import pyfstat
from pyfstat.utils import get_sft_as_arrays

# Local module to simplify plotting
#import tutorial_utils

logger = pyfstat.set_up_logger(label="0_generating_noise", log_level="INFO")
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# NOISE

# Setup Writer
writer_kwargs = {
    "label": "single_detector_gaussian_noise",
    "outdir": "PyFstat_example_data",
    "tstart": 1238166018,  # Starting time of the observation [GPS time]
    "duration": 365 * 86400,  # Duration [seconds]
    "detectors": "H1",  # Detector to simulate, in this case LIGO Hanford
    "F0": 100.0,  # Central frequency of the band to be generated [Hz]
    "Band": 1.0,  # Frequency band-width around F0 [Hz]
    "sqrtSX": 1e-23,  # Single-sided Amplitude Spectral Density of the noise
    "Tsft": 1800,  # Fourier transform time duration
    "SFTWindowType": "tukey",  # Window function to compute short Fourier transforms
    "SFTWindowBeta": 0.01,  # Parameter associated to the window function
}
writer = pyfstat.Writer(**writer_kwargs)

# Create SFTs
writer.make_data()

In [ ]:
# noise fourier data creation

frequency_noise, timestamps_noise, fourier_data_noise = get_sft_as_arrays(writer.sftfilepath)

In [ ]:
# SIGNAL + NOISE

writer_kwargs = {
    "label": "single_detector_gaussian_noise",
    "outdir": "PyFstat_example_data",
    "tstart": 1238166018,
    "duration": 365 * 86400,
    "detectors": "H1",
    "Band": 1.0,  # Frequency band-width around F0 [Hz]
    "sqrtSX": 1e-23,
    "Tsft": 1800,
    "SFTWindowType": "tukey",
    "SFTWindowBeta": 0.01,
}

signal_parameters = {
    "F0": 100.0,
    "F1": -1e-9,
    "Alpha": 0.0,
    "Delta": 0.0,
    "h0": 1e-22,
    "cosi": 1,
    "psi": 0.0,
    "phi": 0.0,
    "tref": writer_kwargs["tstart"],
}

writer = pyfstat.Writer(**writer_kwargs, **signal_parameters)
writer.make_data()

In [ ]:
# Signal + noise fourier data creation

frequency_signal, timestamps_signal, fourier_data_signal = get_sft_as_arrays(writer.sftfilepath)

In [ ]:
# complex SFFT to real data  $ flattening of array

      # signal - 
signal_flatten=np.absolute(fourier_data_signal['H1']).flatten()
signal_matrix=np.absolute(fourier_data_signal['H1'])

     # noise - 
noise_flatten=np.absolute(fourier_data_noise['H1']).flatten()
noise_matrix=np.absolute(fourier_data_noise['H1'])

In [ ]:
# fourier real matrix conversion to power in db

    # signal - 
    
signal_power = 10 * np.log10(signal_matrix)

    
    # noise -
    
noise_power = 10 * np.log10(noise_matrix)

#signal_power,noise_power

In [ ]:
# Plotting frequency scale with power of signal + noise data

plt.figure()
plt.plot(frequency_signal, signal_power, color='black')
plt.xlabel('Signal Frequency (kHz)')
plt.ylabel('Signal power (dB)')
plt.show()


In [ ]:
## Plotting frequency scale with power of noise data

plt.figure()
plt.plot(frequency_noise, noise_power, color='black')
plt.xlabel('Noise Frequency (kHz)')
plt.ylabel('Noise power (dB)')
plt.show()

In [ ]:
#  creating the dict/map corresponding to frequency with max of noise powerarray matrix with (len = 17520) 

noisemax={}
noisemaxfreq={}
for x in range(0,len(frequency_noise)):
    noisemax[frequency_noise[x]]=noise_power[x].max()
for a in noisemax:
    if noisemax[a]>-210:
        noisemaxfreq[a]=noisemax[a]
noisemaxfreqKeys=noisemaxfreq.keys()   
noisemaxfreqKeyslist=list(noisemaxfreqKeys)
len(noisemaxfreqKeyslist)

In [ ]:
#  creating the dict/map corresponding to frequency with max of signal powerarray matrix with (len = 17520) 

signalmax={}
signalmaxfreq={}
for x in range(0,len(frequency_signal)):
    signalmax[frequency_signal[x]]=signal_power[x].max()
for a in signalmax:
    if signalmax[a]>-210:
        signalmaxfreq[a]=signalmax[a]
signalmaxfreqKeys=signalmaxfreq.keys()
signalmaxfreqKeyslist=list(signalmaxfreqKeys)
len(signalmaxfreqKeyslist)

# Creating 17000+ features and Loading Extracted features from external file  additional_noise+additional_signal

In [ ]:
df1 = pd.DataFrame(fourier_data_noise['H1'].imag)
dfx1=df1.applymap(np.absolute)

In [ ]:
df2 = pd.DataFrame(fourier_data_signal['H1'].imag)
dfx2=df2.applymap(np.absolute)

In [ ]:
cp /kaggle/input/finalfreqfeature/finaladditionalfeaturefreq_2022_12_31_07_09_1672513798 .

In [ ]:
ls -l

In [ ]:
# getting additional feature extracted from finaladditionalfeaturefreq_2022_12_31_07_09_1672513798

dbfile = open('finaladditionalfeaturefreq_2022_12_31_07_09_1672513798', 'rb') 
finalfeature= pickle.load(dbfile)

In [ ]:
dbfile.close()

In [ ]:
additional_noise,additional_signal=finalfeature

In [ ]:
# working on signal data frame d2
    
    # adding and filling additional feature """ signal""" column in signal dataframe
for i in additional_signal:
    if i in set(signalmaxfreq.keys()) and i not in set(noisemaxfreq.keys()):
        dfx2[i]=1
    elif i not in set(signalmaxfreq.keys()) and i in set(noisemaxfreq.keys()):
        dfx2[i]=0
    else:
        dfx2[i]=-1

In [ ]:
# working on signal data frame d2
        
    # adding and filling additional feature """noise""" column in signal dataframe d2
for i in additional_noise:
    if i in set(noisemaxfreq.keys()) and i not in set(signalmaxfreq.keys()):
        dfx2[i]=0
    elif i not in set(noisemaxfreq.keys()) and i in set(signalmaxfreq.keys()):
        dfx2[i]=1
    else:
        dfx2[i]=-1    

In [ ]:
# Working on noise data frame d1

    #  adding and filling additional feature """signal""" column in noise dataframe d1
for i in additional_signal:
    if i in set(signalmaxfreq.keys()) and i not in set(noisemaxfreq.keys()):
        dfx1[i]=0
    elif i not in set(signalmaxfreq.keys()) and i in set(noisemaxfreq.keys()):
        dfx1[i]=1
    else:
        dfx1[i]=-1

In [ ]:
# Working on noise data frame d1
    
    #  adding and filling additional feature """noise""" column in noise dataframe d1
for i in additional_noise:
    if i in set(noisemaxfreq.keys()) and i not in set(signalmaxfreq.keys()):
        dfx1[i]=1
    elif i not in set(noisemaxfreq.keys()) and i in set(signalmaxfreq.keys()):
        dfx1[i]=0
    else:
        dfx1[i]=-1

In [ ]:
dfx2['y']=1
dfx1['y']=0

In [ ]:
dfx3=dfx1.append(dfx2, ignore_index=True)

In [ ]:
y_test=dfx3['y']
x_test=dfx3.drop(['y'], axis=1)

# Prediction using PERCEPTRON

In [ ]:
#y_test_pred = ppn.predict(x_test)

In [ ]:
#print('Misclassified examples: %d' % (y_test != y_test_pred).sum())

In [ ]:
# checking loading of perceptron saved model
load_ppn = open('/kaggle/working/perceptron_ppn-2023_01_01_12_30_1672576249', 'rb') 
load_model_ppn = pickle.load(load_ppn)

In [ ]:
load_ppn.close()

In [ ]:
y_test_pred2 = load_model_ppn.predict(x_test)

In [ ]:
print('Misclassified count: %d' % (y_test != y_test_pred2).sum())

# Using KNN

In [ ]:
#from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#classifier= KNeighborsClassifier(n_neighbors=4, metric='minkowski', p=2 )  
#classifier.fit(x, y)  

In [ ]:
#y_pred_knn= classifier.predict(x_test)  

In [ ]:
#print('Misclassified examples: %d' % (y_test != y_pred_knn).sum())

# SVM

In [ ]:
#from sklearn.svm import SVC

In [ ]:
#svm = SVC(kernel='linear', C=1.0, random_state=1)

In [ ]:
#svm.fit(x, y)

In [ ]:
#y_svm_pred=svm.predict(x_test)

In [ ]:
#print('Misclassified examples: %d' % (y_test != y_svm_pred).sum())

In [ ]:
# saving the svm model
#!touch svm_model-`date +"%Y_%m_%d_%I_%M_%s"`

In [ ]:
#ls -l

In [ ]:
# code for saving the perceptron model ppn
    #svm_model_file = open('svm_model-2023_01_01_01_03_1672578238','ab')
    #pickle.dump(svm,svm_model_file)  

In [ ]:
#svm_model_file.close()

In [ ]:
# loading the svm model
load_svm = open('/kaggle/working/svm_model-2023_01_01_01_03_1672578238', 'rb') 
load_model_svm = pickle.load(load_svm)

In [ ]:
load_svm.close()

In [ ]:
y_load_svm=load_model_svm.predict(x_test)

In [ ]:
print('Misclassified examples: %d' % (y_test != y_load_svm).sum())